In [1]:
from pyspark.sql.functions import countDistinct
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SQLContext

In [7]:
def repartition_data(year,month_dict,data_source):
    #read,process all of the click data
    for y in year:
        for m in xrange(3,4):
            for d in xrange(30,32):
                for key,value in  month_dict.iteritems():
                    if key == m:
                        month_name = value 
                date = str(y)+'-'+str(m)+'-'+str(d)
                print date
                try:
                    new_data_path = sqlContext.read.parquet("gs://ds-url-catag/plenty_stickers_data/"+data_source+"/repartioned_data/"+str(y)+'/'+month_name+'/'+date+'.parquet')
                except Exception,e:
                    path = 'gs://ds-url-catag/plenty_stickers_data/'+data_source+'/year='+str(y)+'/month='+str(m)+'/day='+str(d)+'/*'
                    print path
                    try:
                        original_file = spark.read.parquet(path)
                        status =  "File exists"
                    except Exception,e:
                        status =  "Nope"
                    if status == "File exists":
                        print status
                        new_file = original_file.repartition(1)
                        write_path = "gs://ds-url-catag/plenty_stickers_data/"+data_source+"/repartioned_data/"+str(y)+'/'+month_name+'/'+date+'.parquet'
                        print write_path
                        new_file.dropDuplicates().write.mode('overwrite').parquet(write_path)
    return "Completed"
    

In [9]:
if __name__ == '__main__':
    Sticker_datasources = ['event=BBM-STICKER-RECEIVED']
    # values for Sticker_Datasources
    # 'event=BBM-STICKER-RECEIVED','event=BBM-STICKER_SPONSORED_LINK-CLICK','event=BBM-STICKER-CLICK','event=BBM-STICKER-DOWNLOAD','event=BBM-STICKER-SEND'
    year= [2018]
    month_dict = {1: "January",2: "February",3 : "march",4 :"April",5 :"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}
    for ds in Sticker_datasources:
         repartition_data(year,month_dict,ds)